In [15]:
import numpy as np
from pihm import read_output
from pihm import read_mesh
from my_funcs import read_cycles

In [16]:
NA, FRSD, CORN, HAY, SOYB, UROS, ALFA = range(7)

num_elem, _, tri, x, y, _, zmax = read_mesh('..', 'WE38')
_, mgmt = read_cycles('WE38')

# Calculate grid areas
area = [0.5 * ((x[t[1]] - x[t[0]]) * (y[t[2]] - y[t[0]]) - (y[t[1]] - y[t[0]]) * (x[t[2]] - x[t[0]])) for t in tri]
area = np.array(area)

simulations = [
    'WE38',
    'WE38_1dot25xN',
    'WE38_1dot5xN',
    'WE38_lowland',
    'WE38_lowland_manure',
    'WE38_upland',
    'WE38_upland_manure',
]

n_fluxes = [
    "n_auto",
    "n_fix",
    "n_fert",
    'volatil',
    'denitrif',
    "n2o_denitrif",
    "n2o_nitrif",
    'n_harvest',
]


for simulation in simulations:
    print(simulation)

    flux_vals = {}
    ag_flux_vals = {}

    for nf in n_fluxes:
        _, var, _, _ = read_output('..', 'WE38', simulation, nf)
        var = np.mean(var, axis=0) * 1000.0 * 365.0     # Convert to kg/ha/yr
        flux_vals[nf] = np.average(var, weights=area)
        ag_flux_vals[nf] = np.average(var[mgmt != FRSD], weights=area[mgmt != FRSD])

    calculated_fluxes = {
        'Fixation and depostion': lambda x: x['n_fix'] + x['n_auto'],
        'Fertilization': lambda x: x['n_fert'],
        'Volatilization': lambda x: x['volatil'],
        'denitrif_n2': lambda x: x['denitrif'] - x['n2o_denitrif'],
        'denitrif_n2o': lambda x: x['n2o_denitrif'],
        'nitrif': lambda x: x['n2o_nitrif'],
        'Harvest': lambda x: x['n_harvest'],
    }

    for nf in calculated_fluxes:
        print('%s = %.2f (%.2f)' % (nf, calculated_fluxes[nf](flux_vals), calculated_fluxes[nf](ag_flux_vals)))


    _, _no3_mass, _, _ = read_output('..', 'WE38', simulation, 'river.NO3')
    _, _nh4_mass, _, _ = read_output('..', 'WE38', simulation, 'river.NH4')
    _, _stage, _, _ = read_output('..', 'WE38', simulation, 'river.stage')
    _, _q, _, _ = read_output('..', 'WE38', simulation, 'river.flx1')

    stage = _stage[:, 0]
    q = _q[:, 0]

    no3_mass = _no3_mass[:, 0]
    nh4_mass = _nh4_mass[:, 0]

    no3_conc = no3_mass / stage * 100.0
    nh4_conc = nh4_mass / stage * 100.0
    no3_leaching = q * no3_conc * 86400.0 * 1.0E-3 * 365.0 / np.sum(area) * 1.0E4
    nh4_leaching = q * nh4_conc * 86400.0 * 1.0E-3 * 365.0 / np.sum(area) * 1.0E4
    print("no3 leaching = %.2f" % (np.average(no3_leaching)))
    print("nh4 leaching = %.2f" % (np.average(nh4_leaching)))


WE38
Fixation and depostion = 44.96 (51.44)
Fertilization = 85.00 (146.96)
Volatilization = 9.38 (13.78)
denitrif_n2 = 8.09 (10.65)
denitrif_n2o = 0.29 (0.49)
nitrif = 0.55 (0.69)
Harvest = 77.34 (133.73)
no3 leaching = 22.63
nh4 leaching = 0.07
WE38_1dot25xN
Fixation and depostion = 41.88 (45.71)
Fertilization = 106.24 (183.70)
Volatilization = 10.60 (15.89)
denitrif_n2 = 10.21 (14.33)
denitrif_n2o = 0.45 (0.76)
nitrif = 0.60 (0.79)
Harvest = 83.72 (144.76)
no3 leaching = 29.82
nh4 leaching = 0.08
WE38_1dot5xN
Fixation and depostion = 38.68 (40.17)
Fertilization = 127.49 (220.44)
Volatilization = 11.81 (17.97)
denitrif_n2 = 12.62 (18.50)
denitrif_n2o = 0.63 (1.08)
nitrif = 0.66 (0.89)
Harvest = 89.10 (154.06)
no3 leaching = 38.67
nh4 leaching = 0.09
WE38_lowland
Fixation and depostion = 35.11 (34.42)
Fertilization = 106.24 (183.70)
Volatilization = 10.03 (14.89)
denitrif_n2 = 11.89 (17.22)
denitrif_n2o = 0.55 (0.94)
nitrif = 0.63 (0.84)
Harvest = 75.03 (129.73)
no3 leaching = 30.82
nh